In [1]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
import matplotlib.pyplot as plt
%matplotlib inline
import numpy as np
import glob
import os


In [2]:
os.environ["CUDA_VISIBLE_DEVICES"] = "0"
config = tf.compat.v1.ConfigProto()
config.gpu_options.allow_growth=True      #按需调配GPU

In [3]:
#config = tf.compat.v1.ConfigProto(allow_soft_placement=True)
#sess =tf.compat.v1.Session(config=config)

In [4]:
(train_images,train_labels),(_, _) = tf.keras.datasets.mnist.load_data()

In [5]:
train_images.shape,train_images.dtype

((60000, 28, 28), dtype('uint8'))

In [6]:
train_images = train_images.astype('float32')

In [7]:
train_images = train_images.reshape(train_images.shape[0], 28, 28, 1)


In [8]:
train_images.shape,train_images.dtype

((60000, 28, 28, 1), dtype('float32'))

In [9]:
train_images = (train_images - 127.5)/127.5    #归一化到-1到1之间

In [10]:
BATCH_SIZE = 256
BUFFER_SIZE = 60000

In [11]:
datasets = tf.data.Dataset.from_tensor_slices(train_images)

In [12]:
datasets

<TensorSliceDataset shapes: (28, 28, 1), types: tf.float32>

In [13]:
datasets = datasets.shuffle(BUFFER_SIZE).batch(BATCH_SIZE)   #创建输入管道

In [14]:
datasets

<BatchDataset shapes: (None, 28, 28, 1), types: tf.float32>

In [15]:
def generator_model():
    model = tf.keras.Sequential()
    model.add(layers.Dense(7*7*256, use_bias=False, input_shape=(100,)))
    model.add(layers.BatchNormalization())
    model.add(layers.LeakyReLU())

    model.add(layers.Reshape((7, 7, 256)))
  

    model.add(layers.Conv2DTranspose(128, (5, 5), strides=(1, 1), padding='same', use_bias=False))
    model.add(layers.BatchNormalization())
    model.add(layers.LeakyReLU())

    model.add(layers.Conv2DTranspose(64, (5, 5), strides=(2, 2), padding='same', use_bias=False))
    model.add(layers.BatchNormalization())
    model.add(layers.LeakyReLU())

    model.add(layers.Conv2DTranspose(1, (5, 5), strides=(2, 2), padding='same', use_bias=False, activation='tanh'))


    return model

In [16]:
def discriminator_model():
    model = tf.keras.Sequential()
    model.add(layers.Conv2D(64, (5, 5), strides=(2, 2), padding='same',
                                        input_shape=(28,28,1)))
    model.add(layers.LeakyReLU())
    model.add(layers.Dropout(0.3))

    model.add(layers.Conv2D(128, (5, 5), strides=(2, 2), padding='same'))
    model.add(layers.LeakyReLU())
    model.add(layers.Dropout(0.3))
    
    model.add(layers.Conv2D(256, (5, 5), strides=(2, 2), padding='same'))
    model.add(layers.LeakyReLU())
   

    model.add(layers.Flatten())
    model.add(layers.Dense(1))

    return model

In [17]:
cross_entropy = tf.keras.losses.BinaryCrossentropy(from_logits=True)

In [18]:
def discriminator_loss(real_out,fake_out):
    
    #这里希望让真实图像被判定为1,假的图像判定为0
    real_loss = cross_entropy(tf.ones_like(real_out),real_out)
    fake_loss = cross_entropy(tf.zeros_like(fake_out),fake_out)
    
    return real_loss + fake_loss

In [19]:
#生成器接收假的的图片并希望能够被判为真
def generator_loss(fake_out):
    return cross_entropy(tf.ones_like(fake_out),fake_out)

In [20]:
generator_optimizer = tf.keras.optimizers.Adam(1e-4)
discriminator_optimizer = tf.keras.optimizers.Adam(1e-4)   #创建优化器

In [21]:
EOPCHS = 10
NOISE_DIM = 100

num_exp_to_generator = 16  #生成观察样本

seed = tf.random.normal([num_exp_to_generator,NOISE_DIM]) #生成16个长度为100的随机向量

In [22]:
generator = generator_model()#之前定义的是函数，现在要创建模型

In [23]:
discriminator = discriminator_model()

In [24]:
generator.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 12544)             1254400   
_________________________________________________________________
batch_normalization (BatchNo (None, 12544)             50176     
_________________________________________________________________
leaky_re_lu (LeakyReLU)      (None, 12544)             0         
_________________________________________________________________
reshape (Reshape)            (None, 7, 7, 256)         0         
_________________________________________________________________
conv2d_transpose (Conv2DTran (None, 7, 7, 128)         819200    
_________________________________________________________________
batch_normalization_1 (Batch (None, 7, 7, 128)         512       
_________________________________________________________________
leaky_re_lu_1 (LeakyReLU)    (None, 7, 7, 128)         0

In [25]:
discriminator.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 14, 14, 64)        1664      
_________________________________________________________________
leaky_re_lu_3 (LeakyReLU)    (None, 14, 14, 64)        0         
_________________________________________________________________
dropout (Dropout)            (None, 14, 14, 64)        0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 7, 7, 128)         204928    
_________________________________________________________________
leaky_re_lu_4 (LeakyReLU)    (None, 7, 7, 128)         0         
_________________________________________________________________
dropout_1 (Dropout)          (None, 7, 7, 128)         0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 4, 4, 256)        

In [26]:
@tf.function
def train_step(images):
    noise = tf.random.normal([BATCH_SIZE,NOISE_DIM])
    with tf.GradientTape() as gen_tape,tf.GradientTape() as disc_tape:
         
        real_out = discriminator(images,training=True)
        gen_image = generator(noise,training=True)
        fake_out = discriminator(gen_image,training=True)
         
        gen_loss = generator_loss(fake_out)
        disc_loss = discriminator_loss(real_out,fake_out)
        
    gradient_gen = gen_tape.gradient(gen_loss,generator.trainable_variables)
    gradient_disc = disc_tape.gradient(disc_loss,discriminator.trainable_variables)
    
    generator_optimizer.apply_gradients(zip(gradient_gen,generator.trainable_variables))
    discriminator_optimizer.apply_gradients(zip(gradient_disc,discriminator.trainable_variables))
    

In [27]:
#画出这一个批次的图片
def generate_plot_image(gen_model,test_noise):
    pre_images = gen_model(test_noise,training=False)
    fig = plt.figure(figsize=(4,4))  #创建画布4*4单位是英寸
    for i in range(pre_images.shape[0]):
        plt.subplot(4,4,i+1)#4行4列 依次填入图像
        #因为在生成图片最后用激活函数tanh把图片映射到[-1,1]之间,现在要把取值范围改变成[0,1]的范围
        plt.imshow(pre_images[i,:,:,0] + 1/ 2,cmap='gray')
        plt.axis('off')
    plt.show()

In [28]:
def train(dataset,epochs):
    for epoch in range(epochs):
        print('Epoch',epoch + 1,':')
        for image_path in dataset:
            train_step(image_path)
            print('.',end='')
            generate_plot_image(generator,seed)
        print('\n')
train(datasets,EOPCHS)

Epoch 1 :


KeyboardInterrupt: 

In [ ]:
!nvidia-smi